In [ ]:
import graphviz as gv
import math
import numpy as np

pages listed above.
dot
neato
smyrna
lefty
dotty

In [ ]:
g = gv.Graph(engine='neato')
for ix in range(7):
    g.edge(str(ix), str(ix+1))

g.edge('7', '0')
g.edge('5', '7')
g.edge('1', '4')
g.edge('0', '3')
#g.edge('2', '6')


g

In [ ]:
g.source

1. dfs-tree(node, visited=None)
  1. visited = [False]*n if visited is None
  1. for nbr in nbrhd(node)
      1. if(!visited[nbr])
        1. visited[nbr] = True
        1. dfs.edge(node -> snbr)
        1. dfs-tree(node, nbr)

```python
def post_order(dg, root):
    po_children = [post_order(dg, c) for c in root.children].merge()
    return po_children + [root]
```

dfs_tree = dfs-tree()  


In [ ]:
def graph_repr(graph):
    str_data = " "*4
    for row in range(graph.num_vertices):
        str_data += " " + str(row)
    str_data += "\n"+" "*4+"--"*(graph.num_vertices+1)+"\n"

    for row in range(graph.num_vertices):
        str_data += str(row) + " | "
        for col in range(graph.num_vertices):
            str_data += " " + ("1" if graph.data[row,col] else "0")
        str_data += "\n"
    return str_data

In [ ]:
class Graph:
    def __init__(self, _num_vertices):
        self.data = np.zeros((_num_vertices, _num_vertices), dtype=bool)
        self.num_vertices = _num_vertices
    def assert_bounds(self, node):
        assert(node >=0 and node<self.num_vertices)
    def add_edge(self, node1, node2):
        self.assert_bounds(node1); self.assert_bounds(node2)
        self.data[node1, node2] = self.data[node2, node1] = True
    def __str__(self):
        return self.__repr__()
    def nbrs(self, node):
        for nbr in range(self.num_vertices):
            if self.data[node, nbr]:
                yield nbr

Graph.__repr__ = graph_repr   
        
num_vertices = 8
g1 = Graph(num_vertices)
for ix in range(num_vertices-1):
    g1.add_edge(ix, ix+1)
print(", ".join([str(nbr) for nbr in g1.nbrs(1)]))
g1

In [ ]:
class DGraph(Graph):
    def __init__(self, num_vertices):
        Graph.__init__(self, num_vertices)
        self.root = None
    def add_edge(self, node1, node2):
        self.data[node1, node2] = True
    def is_edge(self, node1, node2):
        return self.data[node1, node2]
    def post_order(self, node=None):
        if node is None:
            assert self.root is not None, "root is none"
            node = self.root
        post_order_nodes = []
        for nbr in self.nbrs(node):
            post_order_nodes += self.post_order(nbr)
        return post_order_nodes + [node]
    def pre_order(self, node=None):
        if node is None:
            assert self.root is not None, "root is none"
            node = self.root
        pre_order_nodes = [node]
        for nbr in self.nbrs(node):
            pre_order_nodes += self.pre_order(nbr)
        return pre_order_nodes
    def form_num_descendants(self, node=None):
        if node is None:
            node = self.root
            self.nd = [0 for _ in range(self.num_vertices)]
        self.nd[node] += 1
        for nbr in self.nbrs(node):
            self.form_num_descendants(nbr)
            self.nd[node] += self.nd[nbr]
    def print_viz(self, engine='neato'):
        g_viz = gv.Digraph(engine=engine)
        for node in range(self.num_vertices):
            for nbr in self.nbrs(node):
                g_viz.edge(str(node), str(nbr))
        return g_viz
        
    
dg = DGraph(8)
dg.add_edge(3,4)
print(", ".join([str(nbr) for nbr in dg.nbrs(4)]))
print(", ".join([str(nbr) for nbr in dg.nbrs(3)]))
dg

In [ ]:
class DfsTreeGenerator:
    def __init__(self, graph, node):
        self.num_vertices = graph.num_vertices
        self.dfs_tree = DGraph(graph.num_vertices)
        self.visited = [False for _ in range(self.num_vertices)]
        self.dfs_tree.root = node
        self.form_dfs_tree(graph, node)
    def form_dfs_tree(self, graph, node):
        self.visited[node] = True
        for nbr in graph.nbrs(node):
            if(not self.visited[nbr]):
                self.dfs_tree.add_edge(node, nbr)
                self.form_dfs_tree(graph, nbr)
    def get(self):
        return self.dfs_tree

print(g1)
dgraph_dfs = DfsTreeGenerator(g1, 1).get()
print(dgraph_dfs)
print(dgraph_dfs.post_order())

In [ ]:
num_vertices = 8
g2 = Graph(num_vertices)
for ix in range(num_vertices-1):
    g2.add_edge(ix, ix+1)
g2.add_edge(7, 0)
g2.add_edge(0, 3)
g2.add_edge(1, 4)
g2.add_edge(2, 6)
g2.add_edge(5, 7)
print(", ".join([str(nbr) for nbr in g2.nbrs(1)]))
print(g2)

dfs_tree = DfsTreeGenerator(g2, 1).get()
print(dfs_tree)
dfs_tree.print_viz()

In [ ]:
print(dfs_tree.pre_order())
print(dfs_tree.post_order())

In [ ]:
def get_numbers(dfs_tree):
    post_order_dfs = dfs_tree.post_order()
    numbers_machi = [ix for ix in range(dfs_tree.num_vertices)]
    for ix, val in enumerate(post_order_dfs):
        numbers_machi[val] = ix
    return numbers_machi
numbers = get_numbers(dfs_tree)
print(numbers)

In [ ]:
dfs_tree.form_num_descendants()
dfs_tree.nd

In [ ]:
dfs_tree_gv = dfs_tree.print_viz('neato')
# body
dfs_tree_gv.body.append('size = "7.75,1.25"')
dfs_tree_gv

Dependencies
* TarjanTree
  * form_dfs_tree -> DfsTreeGenerator(.)
  * numbers -> DGraph.post_order()
  * ND -> DGraph.nbrs(.)

In [ ]:
class TarjanTree:
    def __init__(self, graph, root):
        self.num_vertices = graph.num_vertices
        self.root = root
        self.l = [None for _ in range(self.num_vertices)]
        self.h = [None for _ in range(self.num_vertices)]
        self.bridges = []
        self.do_things(graph, root)
        
    def do_things(self, graph, root):
        self.dfs_tree = self.form_dfs_tree(graph, root)
        self.numbers = self.get_numbers(self.dfs_tree)
        self.form_num_descendants(self.dfs_tree)
        self.back_edges = self.compute_back_edges(graph, self.dfs_tree)
        self.compute_lh()
        self.bridges = self.compute_bridges()
    def compute_bridges(self):
        for node in range(self.num_vertices):
            for nbr in self.dfs_tree.nbrs(node):
                h_check = self.h[nbr] <= self.numbers[nbr]
                l_check = self.l[nbr] > self.numbers[nbr] - self.nd[nbr]
                if h_check and l_check:
                    self.bridges.append([node,nbr])
    def compute_l(self, node):
        if self.l[node] is not None:
            return self.l[node]
        terms =  []
        # v - ND(V) + 1
        terms += [self.numbers[node] - self.nd[node] + 1]
        # L(w) | v -> w
        terms += [self.compute_l(nbr) for nbr in self.dfs_tree.nbrs(node)]
        # w | v -- w
        terms += [self.numbers[nbr_non] for nbr_non, val in enumerate(self.back_edges[node,:]) if val]
        self.l[node] = min(terms)
        return self.l[node]
    def compute_h(self, node):
        if self.h[node] is not None:
            return self.h[node]
        terms =  []
        # v
        terms += [self.numbers[node]]
        # L(w) | v -> w
        terms += [self.compute_h(nbr) for nbr in self.dfs_tree.nbrs(node)]
        # w | v -- w
        terms += [self.numbers[nbr_non] for nbr_non, val in enumerate(self.back_edges[node,:]) if val]
        self.h[node] = max(terms)
        return self.h[node]
    def compute_lh(self):
        self.compute_l(self.root)
        self.compute_h(self.root)
    def get_numbers(self, dfs_tree):
        post_order_dfs = dfs_tree.post_order()
        numbers_machi = [ix for ix in range(dfs_tree.num_vertices)]
        for ix, val in enumerate(post_order_dfs):
            numbers_machi[val] = ix
        return numbers_machi
    def form_dfs_tree(self, graph, node):
         return DfsTreeGenerator(graph, node).get()
    def form_num_descendants(self, dfs_tree, node=None):
        if node is None:
            node = self.root
            self.nd = [0 for _ in range(self.num_vertices)]
        self.nd[node] += 1
        for nbr in dfs_tree.nbrs(node):
            self.form_num_descendants(dfs_tree, nbr)
            self.nd[node] += self.nd[nbr]
    def compute_back_edges(self, graph, dfs_tree):
        back_edges = np.zeros((graph.num_vertices,graph.num_vertices), dtype=bool)
        for node in range(graph.num_vertices):
            for nbr in graph.nbrs(node):
                if dfs_tree.is_edge(node, nbr) or dfs_tree.is_edge(nbr,node):
                    continue
                back_edges[node,nbr] = True
                #if node<nbr:
                #    back_edges.append([node,nbr])
        return back_edges
    def get_gviz(self, with_backedges=True, graph_attrs=[]):
        dfs_viz = dfs_tree.print_viz('dot')
        self.annotate_dfs_viz(dfs_viz, with_backedges)
        for graph_attr in graph_attrs:
            dfs_viz.body.append(graph_attr)
        return dfs_viz
    def annotate_dfs_viz(self, dfs_viz, with_backedges=True):
        for node in range(self.num_vertices):
            dfs_viz.node(str(node), 
                         label="{0}\nN:{1}\nND:{2}".format(node,
                                                      self.numbers[node],
                                                      self.nd[node]))
        if with_backedges:
            for row in range(self.num_vertices):
                for col in range(row, self.num_vertices): 
                    if not self.back_edges[row, col]:
                        continue
                    dfs_viz.edge(str(row),str(col), color='red')
tt = TarjanTree(graph=g2, root=1)
print('NUMBER:', tt.numbers)
print('ND:', tt.nd)
print(tt.bridges)
tt.get_gviz(with_backedges=False, graph_attrs=['size="5,5"','rankdir="LR"'])

In [ ]:
tt.get_gviz(with_backedges=True, graph_attrs=['size="5,5"','rankdir="LR"'])

In [ ]:
tt.back_edges

In [ ]:
tt.back_edges[1,:]

In [ ]:
[ix for (ix, val) in enumerate(tt.back_edges[1,:]) if val]

In [ ]:
[val for val in tt.back_edges[1,:] if val]